In [2]:
%matplotlib inline
import h5py
import numpy as np
import os
import pandas as pd
#import librosa
import matplotlib.pyplot as plt

In [3]:
bsfldr = 'dataset'#the basefolder to the dataset
#not using anywhere
seq_len = 5 #sequence of input length in seconds, this length will be the same for audio spectrogram as well

#this is for video data
in_fps = 30 #number of frames input per second
out_fps = 10 #subsample the frames
frame_sz = (128, 128) #the size of the image

#audio feature, these params are picked from voxceleb paper
sfft_wl = 0.025 #spectrogram window in seconds
sfft_hop = 0.010 #hop length in seconds 
n_fft = 1024 # the size of the dataset is t x (n_fft/2+1)

In [8]:
def get_all_files(basefldr):
    
    # Get the list of all files in directory tree at given path
    listOfFiles = {}
    dirnames = [f for f in os.listdir(basefldr) if os.path.isdir(os.path.join(basefldr, f))]#all the subject names
 
    subject_lbl = pd.DataFrame() #store the mapping of labels and subjects
    subject_lbl['subjects'] = dirnames
    subject_lbl['label'] = np.arange(len(dirnames))
    subject_lbl.to_csv('subject_label.csv')
    
    i = 0
    for d in dirnames:
        listOfFiles[i] = np.sort(np.array([os.path.join(basefldr, d, 'real', os.path.splitext(f)[0]) 
                                  for f in os.listdir(os.path.join(basefldr, d, 'real')) if f.endswith('.csv')]))
        i = i+1
        
    return listOfFiles
    
file_paths = get_all_files(bsfldr)

In [9]:
print(file_paths )

{0: array(['dataset/biden/real/-O_GzylneSo_0',
       'dataset/biden/real/-O_GzylneSo_1',
       'dataset/biden/real/-O_GzylneSo_2',
       'dataset/biden/real/-O_GzylneSo_3',
       'dataset/biden/real/0C39h7_wP4c_0',
       'dataset/biden/real/0C39h7_wP4c_1',
       'dataset/biden/real/0C39h7_wP4c_10',
       'dataset/biden/real/0C39h7_wP4c_2',
       'dataset/biden/real/0C39h7_wP4c_3',
       'dataset/biden/real/0C39h7_wP4c_4',
       'dataset/biden/real/0C39h7_wP4c_5',
       'dataset/biden/real/0C39h7_wP4c_6',
       'dataset/biden/real/0C39h7_wP4c_7',
       'dataset/biden/real/0C39h7_wP4c_8',
       'dataset/biden/real/0C39h7_wP4c_9',
       'dataset/biden/real/0O9Cd6vjdN0_0',
       'dataset/biden/real/0O9Cd6vjdN0_1',
       'dataset/biden/real/0O9Cd6vjdN0_10',
       'dataset/biden/real/0O9Cd6vjdN0_11',
       'dataset/biden/real/0O9Cd6vjdN0_12',
       'dataset/biden/real/0O9Cd6vjdN0_13',
       'dataset/biden/real/0O9Cd6vjdN0_14',
       'dataset/biden/real/0O9Cd6vjdN0_15',


In [10]:
def get_landmark(csv_file, verbose=False):
    
    x = np.array(csv_file.loc[:, ' X_0':' X_67'])
    y = np.array(csv_file.loc[:, ' Y_0':' Y_67'])
    z = np.array(csv_file.loc[:, ' Z_0':' Z_67'])

    r_x = np.array(csv_file.loc[:, ' pose_Rx'])
    r_y = np.array(csv_file.loc[:, ' pose_Ry'])
    r_z = np.array(csv_file.loc[:, ' pose_Rz'])

    x_new = x * (np.cos(r_z)*np.cos(r_y))[:, np.newaxis] \
            + y * (np.cos(r_z)*np.sin(r_y)*np.sin(r_x) + np.sin(r_z)*np.cos(r_x))[:, np.newaxis] \
            + z * (np.sin(r_z)*np.sin(r_x) - np.cos(r_z)*np.sin(r_y)*np.cos(r_x))[:, np.newaxis]
    y_new = -x * (np.sin(r_z)*np.cos(r_y))[:, np.newaxis] \
            + y * (np.cos(r_z)*np.cos(r_x) - np.sin(r_z)*np.sin(r_y)*np.sin(r_x))[:, np.newaxis] \
            + z * (np.sin(r_z)*np.sin(r_y)*np.cos(r_x) + np.cos(r_z)*np.sin(r_x))[:, np.newaxis]

    y_new = -y_new

    #x_new = x.copy(); y_new = -y.copy()

    #for every row find t_x, t_y, theta, and scale
    l_e_x = np.mean(x_new[:, 36:42], axis=1)
    l_e_y = np.mean(y_new[:, 36:42], axis=1)
    r_e_x = np.mean(x_new[:, 42:48], axis=1)
    r_e_y = np.mean(y_new[:, 42:48], axis=1)

    #translate
    x = x_new - l_e_x[:, np.newaxis]
    y = y_new - l_e_y[:, np.newaxis]
    r_e_x = r_e_x - l_e_x
    r_e_y = r_e_y - l_e_y
    l_e_x = l_e_x - l_e_x
    l_e_y = l_e_y - l_e_y

    #rotate theta, assumption r_e_x is positive
    cos_theta = r_e_x / np.sqrt(r_e_x**2 + r_e_y**2)
    sin_theta = np.sqrt(1 - cos_theta**2)
    sin_theta[r_e_y<0] = -sin_theta[r_e_y<0]

    x_new = x * cos_theta[:, np.newaxis] + y * sin_theta[:, np.newaxis]
    y_new = y * cos_theta[:, np.newaxis] - x * sin_theta[:, np.newaxis]
    x = x_new
    y = y_new
    #for every row find t_x, t_y, theta, and scale
    l_e_x = np.mean(x_new[:, 36:42], axis=1)
    l_e_y = np.mean(y_new[:, 36:42], axis=1)
    r_e_x = np.mean(x_new[:, 42:48], axis=1)
    r_e_y = np.mean(y_new[:, 42:48], axis=1)

    #scale
    x = x / r_e_x[:, np.newaxis]
    y = y / r_e_x[:, np.newaxis]
    l_e_x = l_e_x / r_e_x
    l_e_y = l_e_y / r_e_x
    r_e_y = r_e_y / r_e_x
    r_e_x = r_e_x / r_e_x


    if verbose:
        fig = plt.figure()
        for i in range(len(l_e_y)):
            fig.clf()
            plt.scatter(x[i, :], y[i, :], c='b', marker='.')
            plt.scatter(l_e_x[i], l_e_y[i], c='r', marker='.')
            plt.scatter(r_e_x[i], r_e_y[i], c='r', marker='.')
            fig.canvas.draw()
            plt.show()
            plt.pause(0.001)
            
        plt.close(fig)

    out_ar = dict()
    out_ar['x'] = x
    out_ar['y'] = y

    return out_ar

In [11]:
h5file = h5py.File('landmark.hdf5', 'a')
lbls = list(file_paths.keys()) #hdf5 file will have a separate group for each label
avg_win = int(in_fps/out_fps)
for i in lbls:
    
    if str(i) in h5file.keys():
        group = h5file[str(i)]
    else:
        group = h5file.create_group(str(i)) #create label specific group
    vid_names = file_paths[i]
    
    for v in vid_names:
        
        db_name = v.split('/')[-1] # name of dataset
        if db_name not in group.keys(): #do only if the file has not been processed

            #the number of frames we will get from this video
            full_csv = pd.read_csv(v + '.csv') # read the csv
            
            lndmrk = get_landmark(full_csv, verbose=False)
            x = lndmrk['x']
            y = lndmrk['y']
            aus = np.array(full_csv.loc[:, ' AU01_r':' AU45_r'])
            r_xyz = np.array(full_csv.loc[:, ' pose_Rx':' pose_Rz'])
            
            out_lndmrk = np.concatenate((x, y, aus, r_xyz), axis=1)
            out_frame_cnt = int(np.floor(len(out_lndmrk)*out_fps/in_fps))
            out_lndmrk = out_lndmrk[:(out_frame_cnt*avg_win), :]
            
            out_lndmrk = np.squeeze(np.mean(np.reshape(out_lndmrk, (out_lndmrk.shape[1], avg_win, out_frame_cnt)), 
                                            axis=1)).T
            out_lndmrk = out_lndmrk - np.min(out_lndmrk, axis=0, keepdims=True)
            out_lndmrk = out_lndmrk / np.max(out_lndmrk, axis=0, keepdims=True)
            
            # create the dataset with the complete groups with chunksize 
            data = group.create_dataset(db_name, out_lndmrk.shape, dtype='f4',
                                        compression="gzip", compression_opts=4)
            data[:] = out_lndmrk
            h5file.flush() #write out the file

h5file.close()

In [42]:
# with h5py.File('landmark.hdf5', 'r') as f:
#     for groups in f.keys():
#         print(groups)
#         for keys in groups.keys():
#             print(keys)
        

0


AttributeError: 'str' object has no attribute 'keys'

In [53]:
with h5py.File('landmark.hdf5', 'r') as f:
    g = f.visititems(print)
    
    

0 <HDF5 group "/0" (198 members)>
0/-O_GzylneSo_0 <HDF5 dataset "-O_GzylneSo_0": shape (100, 156), type "<f4">
0/-O_GzylneSo_1 <HDF5 dataset "-O_GzylneSo_1": shape (113, 156), type "<f4">
0/-O_GzylneSo_2 <HDF5 dataset "-O_GzylneSo_2": shape (138, 156), type "<f4">
0/-O_GzylneSo_3 <HDF5 dataset "-O_GzylneSo_3": shape (115, 156), type "<f4">
0/0C39h7_wP4c_0 <HDF5 dataset "0C39h7_wP4c_0": shape (172, 156), type "<f4">
0/0C39h7_wP4c_1 <HDF5 dataset "0C39h7_wP4c_1": shape (200, 156), type "<f4">
0/0C39h7_wP4c_10 <HDF5 dataset "0C39h7_wP4c_10": shape (171, 156), type "<f4">
0/0C39h7_wP4c_2 <HDF5 dataset "0C39h7_wP4c_2": shape (111, 156), type "<f4">
0/0C39h7_wP4c_3 <HDF5 dataset "0C39h7_wP4c_3": shape (158, 156), type "<f4">
0/0C39h7_wP4c_4 <HDF5 dataset "0C39h7_wP4c_4": shape (152, 156), type "<f4">
0/0C39h7_wP4c_5 <HDF5 dataset "0C39h7_wP4c_5": shape (120, 156), type "<f4">
0/0C39h7_wP4c_6 <HDF5 dataset "0C39h7_wP4c_6": shape (111, 156), type "<f4">
0/0C39h7_wP4c_7 <HDF5 dataset "0C39h7_wP

1/IFtsa9r_WUE_28 <HDF5 dataset "IFtsa9r_WUE_28": shape (177, 156), type "<f4">
1/IFtsa9r_WUE_30 <HDF5 dataset "IFtsa9r_WUE_30": shape (112, 156), type "<f4">
1/IFtsa9r_WUE_31 <HDF5 dataset "IFtsa9r_WUE_31": shape (166, 156), type "<f4">
1/IFtsa9r_WUE_35 <HDF5 dataset "IFtsa9r_WUE_35": shape (195, 156), type "<f4">
1/IFtsa9r_WUE_37 <HDF5 dataset "IFtsa9r_WUE_37": shape (108, 156), type "<f4">
1/IFtsa9r_WUE_40 <HDF5 dataset "IFtsa9r_WUE_40": shape (129, 156), type "<f4">
1/IFtsa9r_WUE_46 <HDF5 dataset "IFtsa9r_WUE_46": shape (114, 156), type "<f4">
1/IFtsa9r_WUE_49 <HDF5 dataset "IFtsa9r_WUE_49": shape (103, 156), type "<f4">
1/IFtsa9r_WUE_50 <HDF5 dataset "IFtsa9r_WUE_50": shape (130, 156), type "<f4">
1/IFtsa9r_WUE_54 <HDF5 dataset "IFtsa9r_WUE_54": shape (130, 156), type "<f4">
1/IFtsa9r_WUE_56 <HDF5 dataset "IFtsa9r_WUE_56": shape (185, 156), type "<f4">
1/IFtsa9r_WUE_59 <HDF5 dataset "IFtsa9r_WUE_59": shape (166, 156), type "<f4">
1/IFtsa9r_WUE_61 <HDF5 dataset "IFtsa9r_WUE_61": sha

2/yCTcdtkBbdU_0 <HDF5 dataset "yCTcdtkBbdU_0": shape (449, 156), type "<f4">
2/yCTcdtkBbdU_1 <HDF5 dataset "yCTcdtkBbdU_1": shape (449, 156), type "<f4">
2/yCTcdtkBbdU_2 <HDF5 dataset "yCTcdtkBbdU_2": shape (449, 156), type "<f4">
2/zk5KoCv4REE_0 <HDF5 dataset "zk5KoCv4REE_0": shape (458, 156), type "<f4">
3 <HDF5 group "/3" (367 members)>
3/-MTuj907mHw_12 <HDF5 dataset "-MTuj907mHw_12": shape (304, 156), type "<f4">
3/-MTuj907mHw_14 <HDF5 dataset "-MTuj907mHw_14": shape (575, 156), type "<f4">
3/-MTuj907mHw_18 <HDF5 dataset "-MTuj907mHw_18": shape (488, 156), type "<f4">
3/-MTuj907mHw_2 <HDF5 dataset "-MTuj907mHw_2": shape (310, 156), type "<f4">
3/-MTuj907mHw_20 <HDF5 dataset "-MTuj907mHw_20": shape (495, 156), type "<f4">
3/-MTuj907mHw_22 <HDF5 dataset "-MTuj907mHw_22": shape (492, 156), type "<f4">
3/-MTuj907mHw_26 <HDF5 dataset "-MTuj907mHw_26": shape (139, 156), type "<f4">
3/-MTuj907mHw_6 <HDF5 dataset "-MTuj907mHw_6": shape (124, 156), type "<f4">
3/-MTuj907mHw_8 <HDF5 dataset 

3/sTjI0FdkyTo_40 <HDF5 dataset "sTjI0FdkyTo_40": shape (302, 156), type "<f4">
3/sTjI0FdkyTo_41 <HDF5 dataset "sTjI0FdkyTo_41": shape (287, 156), type "<f4">
3/sTjI0FdkyTo_43 <HDF5 dataset "sTjI0FdkyTo_43": shape (376, 156), type "<f4">
3/sTjI0FdkyTo_44 <HDF5 dataset "sTjI0FdkyTo_44": shape (137, 156), type "<f4">
3/sTjI0FdkyTo_46 <HDF5 dataset "sTjI0FdkyTo_46": shape (240, 156), type "<f4">
3/sTjI0FdkyTo_47 <HDF5 dataset "sTjI0FdkyTo_47": shape (100, 156), type "<f4">
3/sTjI0FdkyTo_48 <HDF5 dataset "sTjI0FdkyTo_48": shape (293, 156), type "<f4">
3/sTjI0FdkyTo_5 <HDF5 dataset "sTjI0FdkyTo_5": shape (1293, 156), type "<f4">
3/sTjI0FdkyTo_6 <HDF5 dataset "sTjI0FdkyTo_6": shape (400, 156), type "<f4">
3/sTjI0FdkyTo_7 <HDF5 dataset "sTjI0FdkyTo_7": shape (351, 156), type "<f4">
3/sTjI0FdkyTo_8 <HDF5 dataset "sTjI0FdkyTo_8": shape (186, 156), type "<f4">
3/u8vsCXN9JMg_10 <HDF5 dataset "u8vsCXN9JMg_10": shape (752, 156), type "<f4">
3/u8vsCXN9JMg_11 <HDF5 dataset "u8vsCXN9JMg_11": shape (215

5/NP4-2bItxY8_8 <HDF5 dataset "NP4-2bItxY8_8": shape (163, 156), type "<f4">
5/NP4-2bItxY8_9 <HDF5 dataset "NP4-2bItxY8_9": shape (130, 156), type "<f4">
5/Qv1R3Pu7LZU_0 <HDF5 dataset "Qv1R3Pu7LZU_0": shape (660, 156), type "<f4">
5/Qv1R3Pu7LZU_1 <HDF5 dataset "Qv1R3Pu7LZU_1": shape (163, 156), type "<f4">
5/Qv1R3Pu7LZU_10 <HDF5 dataset "Qv1R3Pu7LZU_10": shape (555, 156), type "<f4">
5/Qv1R3Pu7LZU_11 <HDF5 dataset "Qv1R3Pu7LZU_11": shape (209, 156), type "<f4">
5/Qv1R3Pu7LZU_12 <HDF5 dataset "Qv1R3Pu7LZU_12": shape (870, 156), type "<f4">
5/Qv1R3Pu7LZU_13 <HDF5 dataset "Qv1R3Pu7LZU_13": shape (685, 156), type "<f4">
5/Qv1R3Pu7LZU_14 <HDF5 dataset "Qv1R3Pu7LZU_14": shape (454, 156), type "<f4">
5/Qv1R3Pu7LZU_15 <HDF5 dataset "Qv1R3Pu7LZU_15": shape (687, 156), type "<f4">
5/Qv1R3Pu7LZU_16 <HDF5 dataset "Qv1R3Pu7LZU_16": shape (711, 156), type "<f4">
5/Qv1R3Pu7LZU_17 <HDF5 dataset "Qv1R3Pu7LZU_17": shape (518, 156), type "<f4">
5/Qv1R3Pu7LZU_18 <HDF5 dataset "Qv1R3Pu7LZU_18": shape (527,

In [52]:
print(g)

None


In [54]:
f = h5py.File('landmark.hdf5', 'r')

In [67]:
labels = list(f.keys())

In [68]:
print(labels)

['0', '1', '2', '3', '4', '5']


In [82]:
# groups=[]
# for x in labels:
#     groups.append(f[x])

In [84]:
# print(groups)

[<HDF5 group "/0" (198 members)>, <HDF5 group "/1" (362 members)>, <HDF5 group "/2" (44 members)>, <HDF5 group "/3" (367 members)>, <HDF5 group "/4" (165 members)>, <HDF5 group "/5" (207 members)>]


In [128]:
# z = groups[0].keys()

In [139]:
# for l in z:
#     print(z)

<KeysViewHDF5 ['-O_GzylneSo_0', '-O_GzylneSo_1', '-O_GzylneSo_2', '-O_GzylneSo_3', '0C39h7_wP4c_0', '0C39h7_wP4c_1', '0C39h7_wP4c_10', '0C39h7_wP4c_2', '0C39h7_wP4c_3', '0C39h7_wP4c_4', '0C39h7_wP4c_5', '0C39h7_wP4c_6', '0C39h7_wP4c_7', '0C39h7_wP4c_8', '0C39h7_wP4c_9', '0O9Cd6vjdN0_0', '0O9Cd6vjdN0_1', '0O9Cd6vjdN0_10', '0O9Cd6vjdN0_11', '0O9Cd6vjdN0_12', '0O9Cd6vjdN0_13', '0O9Cd6vjdN0_14', '0O9Cd6vjdN0_15', '0O9Cd6vjdN0_16', '0O9Cd6vjdN0_17', '0O9Cd6vjdN0_18', '0O9Cd6vjdN0_19', '0O9Cd6vjdN0_2', '0O9Cd6vjdN0_20', '0O9Cd6vjdN0_21', '0O9Cd6vjdN0_22', '0O9Cd6vjdN0_23', '0O9Cd6vjdN0_3', '0O9Cd6vjdN0_4', '0O9Cd6vjdN0_5', '0O9Cd6vjdN0_6', '0O9Cd6vjdN0_7', '0O9Cd6vjdN0_8', '0O9Cd6vjdN0_9', '3Sa8G-VR13Q_0', '3Sa8G-VR13Q_1', '6SzWWsBLpdg_0', '6SzWWsBLpdg_1', '6SzWWsBLpdg_2', '6SzWWsBLpdg_3', '6SzWWsBLpdg_4', '6SzWWsBLpdg_5', '7-lr3iO83RI_1', '8GfZxqjkdrk_0', '8GfZxqjkdrk_1', '8GfZxqjkdrk_10', '8GfZxqjkdrk_11', '8GfZxqjkdrk_12', '8GfZxqjkdrk_13', '8GfZxqjkdrk_14', '8GfZxqjkdrk_15', '8GfZxqjkdrk

In [108]:
# dsets = []
# for label in labels:
#     dsets.append(f[label])

In [109]:
# lst=[]
# for dset in dsets:
#     lst.append(dset.keys())

In [ ]:
labels = list(f.keys())

